## 5.1 层和块
之前首次介绍神经网络时，我们关注的是具有单一输出的线性模型。在这里，整个模型只有一个输出，注意，单个神经网络(1)接受一些输入；(2)生成相应的标量输出；(3)具有一组相关参数(parameters)，更新这些参数可以优化某目标函数。

然后，当考虑具有多个输出的网络时，我们利用矢量化算法来描述整层神经元。像单层神经元一样，层(1)接受一组输入，(2)生成相应的输出，(3)由一组可调整参数描述。当我们使用softmax回归时，一个单层本身就是模型。然后，我们引入了多层感知机，我们仍然可以认为该模型保留了上面所说的基本架构。

对于多层感知机而言，整个模型及其组成层都是这种架构。整个模型接受原始输入(特征)，生成输出(预测)，并包含一些参数(所有组成层的参数集合)。同样，每个单独的层接收输入(由前一层提供)，生成输出(到下一层的输入)，并且具有一组可调参数，这些参数根据从下一层反向穿鼻的信号进行更新。

事实证明，研究讨论“比单个层大”但“比整个模型小”的组件更有价值。例如，在计算机视觉中广泛流行的ResNet-152架构就有数百层，这些层是由层组(groups of layers)的重复模式组成的。这个ResNet架构赢得了2015年的ImageNet和COCO计算机视觉领域比赛的识别和检测任务。目前ResNet架构仍然是许多视觉任务的首选架构。在其他的领域，如NLP和语音，层组以各种重复模式排列的类似架构现在也是普遍存在。

为了实现这些复杂的网络，我们引入了神经网络块的概念。块(block)可以描述单个层、由多个层组成的组件或整个模型本身。使用块进行抽象的一个好处是可以将一些组合成更大的组件，这一过程通常是递归的，如下图所示。通过定义代码来按需生成任意复杂度的块，我们可以通过简洁的代码实现复杂的神经网络。

<div align=center>
<img src='../../pics/5_1_1.jpeg' width='50%'>
</div>

从编程的角度来看，块由类(class)表示。它的任何子类都必须定义一个将其输入转换为输出的前向传播函数，并且必须存储任何必需的参数。注意，有些块不需要任何参数。最后，为了计算梯度，块必须具有反向传播函数。在定义我们自己的块时，由于自动微分提供了一些后端实现，我们只需要考虑前向传播函数和必需的参数。

In [ ]:
import toe